# Tema 6 - Actividad Sumativa 2

- Martínez Ostoa Néstor I. 
- Procesamiento de Lenguaje Natural
- LCD - IIMAS - UNAM
- Octubre 2021

---
Analizar el texto El ramo azul, de Octavio Paz y responder las siguientes preguntas:

In [162]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk import WordNetLemmatizer

import spacy
import spacy_spanish_lemmatizer

import numpy as np
import pandas as pd
from string import punctuation

import plotly.graph_objects as go

In [77]:
f = open("elramoazul.txt", "r")
text = ''
for line in f.readlines(): text += line
print(text[:40])

El ramo azul, un cuento de Octavio Paz




## 1. Tokeniza el texto. ¿Cuántas palabras hay? (sin símbolos de puntuación).

In [78]:
def clean_token(token):
    """
    A cada token le quita caracteres especiales
    """
    if token == '–respondió-':
        return "respondió"
    t = ''
    special_chars = ['–', '-', '¡', '!', '¿', '?']
    for sp in special_chars:
        if sp in token:
            t = token.split(sp)[1]
    return token if t == '' else t

In [79]:
tokens = []
for token in word_tokenize(text):
    if token not in list(punctuation) and token != '“' and token != '”':
        tokens.append(clean_token(token).lower())
print(tokens[:20])
print(f"\nEn el corpus hay: {len(tokens)} palabras")

['el', 'ramo', 'azul', 'un', 'cuento', 'de', 'octavio', 'paz', 'desperté', 'cubierto', 'de', 'sudor', 'del', 'piso', 'de', 'ladrillos', 'rojos', 'recién', 'regados', 'subía']

En el corpus hay: 826 palabras


## 2. Para cada palabra, da el lema y la raíz.

In [114]:
# Lematización
load_model = spacy.load("es_core_news_sm")

tokenized_text = ''
for token in tokens: tokenized_text += token + " "

lemmas = dict()
for token in load_model(tokenized_text):
    lemmas[token.text] = token.lemma_

print("Lemmatizer\n-----------")
for w in list(lemmas.keys())[100:125]:
    print(f"{w} : {lemmas[w]}")


Lemmatizer
-----------
mesón : mesón
tropecé : tropecar
dueño : dueño
sujeto : sujeto
tuerto : tuerto
reticente : reticente
sentado : sentado
sillita : sillita
tule : tule
fumaba : fumar
ojo : ojo
entrecerrado : entrecerrado
voz : voz
ronca : ronco
preguntó : preguntar
dónde : dónde
va : ir
señor : señor
dar : dar
vuelta : vuelta
hace : hacer
mucho : mucho
calor : calor
hum : hum
todo : todo


In [100]:
# Stemming
stemmer = SnowballStemmer("spanish")
stems = dict()
for t in tokens: stems[t] = stemmer.stem(t)
print("Stemming:\n-----------")
for word in list(stems.keys())[100:125]: print(f"{word} : {stems[word]}")

Stemming:
-----------
mesón : meson
tropecé : tropec
dueño : dueñ
sujeto : sujet
tuerto : tuert
reticente : reticent
sentado : sent
sillita : sillit
tule : tul
fumaba : fum
ojo : ojo
entrecerrado : entrecerr
voz : voz
ronca : ronc
preguntó : pregunt
dónde : dond
va : va
señor : señor
dar : dar
vuelta : vuelt
hace : hac
mucho : much
calor : calor
hum : hum
todo : tod


## 3. ¿Cuántas palabras diferentes hay?

In [120]:
all_lemas = set(lemmas.keys())
print(f"Existen {len(all_lemas)} palabaras diferentes")

Existen 435 palabaras diferentes


## 4. Después de lematizar las palabras, ¿cuántas palabras diferentes hay?

In [121]:
unique_lemmas = set()
for key in lemmas.keys():
    unique_lemmas.add(lemmas[key])
    
print(f"Después de lematizar, existen {len(unique_lemmas)} palabras diferentes")

Después de lematizar, existen 383 palabras diferentes


## 5. ¿Cuál es la diversidad léxica del texto dado? (relación de palabras únicas con respecto al número total de palabras)

In [124]:
diversidad_lexica = round(len(unique_lemmas) / len(all_lemas), 4)
print(f"Diversidad léxica: {diversidad_lexica}")

Diversidad léxica: 0.8805


## 6. ¿Cuáles son las 20 palabras léxicas más frecuentes en el texto? ¿Cuál es su frecuencia?

In [177]:
freqs = dict()
for token in tokens:
    if token not in freqs:
        freqs[token] = 0
    freqs[token] += 1

df = pd.DataFrame({
    'word': freqs.keys(),
    'frequency': freqs.values()
})
df = df.sort_values(by="frequency", ascending=False)
top_20 = df[:20]
top_20

,word,frequency
5,de,37
30,la,36
47,me,24
0,el,22
32,y,20
37,no,20
88,los,16
3,un,16
44,a,14
245,ojos,13


## 7. ¿Cuál es el número promedio de palabras por oración?

In [133]:
wc = []

f = open("elramoazul.txt", "r")
for line in f.readlines():
    splitted = line.split(' ')
    if splitted[0] != '\n':
        wc.append(len(splitted))

avg_wc_per_sentence = round(np.mean(wc), 2)
print(f"El número promedio de palabras por oración es {avg_wc_per_sentence}")

El número promedio de palabras por oración es 23.63


## 8. ¿Cuál  es la frecuencia de de sustantivos, adjetivos y verbos en el texto?.

In [148]:
parts_of_speech = set()

nlp = spacy.load("es_core_news_sm")
doc = nlp(tokenized_text)
for token in doc:
    parts_of_speech.add(token.pos_)

pos_dict = dict()
for pos in parts_of_speech:
    pos_dict[pos] = 0

print(parts_of_speech)

{'ADJ', 'DET', 'NOUN', 'VERB', 'AUX', 'PRON', 'PROPN', 'CCONJ', 'NUM', 'ADP', 'ADV', 'SCONJ', 'INTJ'}


In [149]:
doc = nlp(tokenized_text)
for token in doc:
    pos_dict[token.pos_] += 1

In [161]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['NOUN', 'ADJ', 'VERB'], 
    y=[pos_dict['NOUN'], pos_dict['ADJ'], pos_dict['VERB']],
    text=[pos_dict['NOUN'], pos_dict['ADJ'], pos_dict['VERB']], 
    textposition='auto'
))

fig.update_layout(
    title_text="Frecuencia de sustantivos, adjetivos y verbos",
    template='plotly_dark'
)

fig.show()